# MXN442 - Assignment 2 and 3 Project
For Article: `Predicting pharmaceutical prices. Advances based on purchase-level data and machine learning`

## Importing Relevant Libraries

In [1]:
import pandas as pd
import numpy as np

# Importing a progress bar due to how big the catalog dataset are
from tqdm.notebook import tqdm
import math

# Unfortunately, as the catalogs are in Spanish, we need to translate the data dictionary to identify the columns
from deep_translator import GoogleTranslator

# Due to the extensive national catalogs that are going to be translated, 
# I decided to use multithreading to complete the translation to run faster
from concurrent.futures import ThreadPoolExecutor, as_completed

import os
pd.set_option('display.max_columns', None)

## Main Directories and Variables to be used throughout the notebook

In [2]:
# Setting up the main directories
main_folder = os.getcwd()
data_folder = os.path.join(main_folder, 'data')
generated_data_folder = os.path.join(main_folder, 'generated_data')

# Locales for the Google Translator
english = 'en'
spanish = 'es'
german = 'de'

translator = GoogleTranslator(source = spanish, target = english) 

uruguayan_procurement_catalog_file_name = 'ReporteBusquedaCatalogo_290925_222642'
mexican_procurement_catalog_file_name = 'cucop_20250929'
paraguayan_procurement_catalog_file_name = 'catalogo_n5'

uruguayan_procurement_catalog_file_path = os.path.join(data_folder, f'{uruguayan_procurement_catalog_file_name}.xls')
mexican_procurement_catalog_file_path = os.path.join(data_folder, f'{mexican_procurement_catalog_file_name}.xlsx')
paraguayan_procurement_catalog_file_path = os.path.join(data_folder, f'{paraguayan_procurement_catalog_file_name}.csv')

## Functions that will be used in the notebook

In [3]:
# Here are the functions that are used to complete the Spanish to English Translations (if the file doesn't exist)
def translate_column_batch(col_data, batch_size=50): 
    """ Translate a single column using batch translation. Returns: (column_name, translated_values, indices) """ 
    col_name, series = col_data 
    non_null_mask = series.notna() 
    original_values = series.loc[non_null_mask].astype(str) 
    n = len(original_values) 
    if n == 0: 
        return col_name, [], original_values.index 
    translated_values = [] 
    num_batches = math.ceil(n / batch_size) 

    for i in tqdm(range(num_batches), desc=f"Translating {col_name}", leave=False):
        start = i * batch_size
        end = min((i + 1) * batch_size, n)
        batch = list(original_values.iloc[start:end])
        try:
            translated_batch = translator.translate_batch(batch)
        except Exception:
            translated_batch = []
            for text in tqdm(batch, desc = f"Fallback translation", leave=False):
                try:
                    translated_batch.append(translator.translate(text))
                except:
                    translated_batch.append(text) 
        translated_values.extend(translated_batch)
    return col_name, translated_values, original_values.index

def translate_dataframe_threading(df, columns_to_translate, max_workers = None, batch_size=50):
    df_translated = df.copy()
    col_data_list = [(col, df[col]) for col in columns_to_translate]
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(translate_column_batch, col_data, batch_size): col_data[0] for col_data in col_data_list}
        for f in tqdm(as_completed(futures), total=len(futures), desc="Translating columns"):
            col_name, translated_values, indices = f.result()
            df_translated.loc[indices, col_name] = translated_values
    return df_translated

# This function encapsulates the above functions so that only the arguments need to be passed for each country and their catalogs
def translate_country_procurement_catalog(catalog_df, file_name, file_type, drop_columns = None):
    translated_catalog = catalog_df.copy()

    translation_required = False
    for i in os.listdir(generated_data_folder):
        if f'translated_{file_name}.{file_type}' not in os.listdir(generated_data_folder):
            translation_required = True        
    
    if translation_required:
        print('File Not Found, Running Translation')
        if drop_columns is not None:
            translate_columns = translated_catalog.columns.drop(drop_columns)
        else:
            translate_columns = translated_catalog.columns

        translated_catalog = translate_dataframe_threading(
            catalog_df, translate_columns, max_workers=4, batch_size=50
        )

        translated_catalog.to_csv(os.path.join(generated_data_folder, f'translated_{file_name}.{file_type}'), index = False)
        
    else:
        print('File Found. Loading Dataset')
        translated_catalog = pd.read_csv(os.path.join(generated_data_folder, f'translated_{file_name}.{file_type}'))
    return translated_catalog

# This is used to check what the columns of interest have
def print_columns_check(dataset, country):
    print(f'Procedure Types for Tenders in {country}:', dataset['tender_nationalproceduretype'].value_counts())
    print('')
    print(f'Number of Recorded Bids in {country}:', dataset['tender_recordedbidscount'].value_counts())
    print('')
    print(f'Number of Buyers in {country}: ', dataset['buyer_name'].nunique())
    print('')
    print(f'Number of Bidders in {country}: ', dataset['bidder_name'].nunique())
    print('')
    print(f'Tender Publication Call Dates in {country}: ', dataset['tender_publications_firstcallfor'].value_counts())
    print('')
    print(f'Tender Bid Deadlines in {country}: ', dataset['tender_biddeadline'].value_counts())
    print('')
    print(f'Tender Award Dates in {country}: ', dataset['tender_contractsignaturedate'].value_counts())

## Importing the Datasets

From some later preprocessing, these columns were identified as the most important:

In [4]:
relevant_cols = [
    'tender_id', 'bidder_name', 'tender_year', 'currency',
    'tender_nationalproceduretype', 'tender_supplytype', 'bidder_country',
    'buyer_id', 'bid_price', 'tender_recordedbidscount', 'bidder_id',
    'tender_publications_firstcallfor', 'tender_biddeadline',
    'tender_contractsignaturedate', 'tender_proceduretype',
    'buyer_buyertype', 'tender_title', 'lot_productCode', 'buyer_name',
    'buyer_city', 'buyer_country'
]

In [5]:
mexico_procurement_data = pd.read_csv(os.path.join(data_folder, 'dfid2_mx_210715_csv.csv'))
mexico_procurement_data

,tender_id,bidder_name,tender_year,currency,tender_nationalproceduretype,tender_supplytype,bidder_country,buyer_id,bid_price,tender_recordedbidscount,bidder_id,tender_publications_firstcallfor,tender_biddeadline,tender_contractsignaturedate,tender_proceduretype,buyer_buyertype,tender_title,lot_productCode,buyer_name,buyer_city,buyer_country,buyer_geocodes,bid_priceUsd,tender_indicator_integrity_call_,tender_indicator_integrity_singl,tender_indicator_integrity_tax_h,tender_indicator_integrity_proce,tender_indicator_integrity_adver,tender_indicator_integrity_decis,tender_integrity_winner_ca_share,lot_localProductCode,lot_localProductCode_type
0,236589,vima suministros industriales,2012,MXN,Invitación a Cuando Menos 3 Personas,goods,MX,876.0,4.595500e+04,NaN,7188.0,03aug2012,15aug2012,NaN,invitation (3 entities),Administraciخ Pݢlica Federal,lote refacciones regulador electr nico,99100000,CFE,Monterrey,Mexico,MX161,5847.6533,100,NaN,100.0,100.0,50.0,50.0,95.588844,99100000,CPV2008
1,252386,llantas royal de veracruz sa de cv,2012,MXN,Adjudicación Directa Federal,goods,MX,967.0,1.194000e+04,NaN,NaN,05sep2012,06sep2012,NaN,direct contracting,Administraciخ Pݢlica Federal,ztv 108 12 adquisicion llantas vehiculo 102038,343500005,CFE,Monterrey,Mexico,MX161,1519.3337,100,NaN,100.0,50.0,0.0,50.0,99.981995,34350000,CPV2008
2,240732,praxair mexico s de rl de cv,2012,MXN,Adjudicación Directa Federal,goods,MX,535.0,2.176596e+04,NaN,5423.0,NaN,NaN,NaN,direct contracting,Administraciخ Pݢlica Federal,productos quimicos,99100000,NaN,NaN,NaN,NaN,2769.6616,0,NaN,100.0,0.0,NaN,NaN,98.499550,99100000,CPV2008
3,312547,comercializadora e importadora garflolu sa de cv,2012,MXN,Adjudicación Directa Federal,goods,MX,490.0,1.416830e+05,NaN,NaN,NaN,NaN,NaN,direct contracting,Administraciخ Pݢlica Federal,equipo proteccion,99100000,"IEPSA - Printer and Binder Progreso, S.A. de C.V.",Ciudad de México,Mexico,MX324,18028.7910,0,NaN,100.0,0.0,NaN,NaN,99.960831,99100000,CPV2008
4,214149,instrumentos y equipos falcon sa de cv,2012,MXN,Adjudicación Directa Federal,services,MX,447.0,3.500000e+04,NaN,3823.0,08jul2012,09jul2012,NaN,direct contracting,Administraciخ Pݢlica Federal,mantenimiento preventivo para analizador,99200000,Cinvestav,Ciudad de México,Mexico,MX324,4453.6582,100,NaN,100.0,50.0,0.0,50.0,99.609375,99200000,CPV2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231921,1681324,grupo emequr,2018,MXN,Adjudicación Directa Federal,goods,MX,NaN,7.203800e+04,NaN,6126.0,NaN,NaN,NaN,direct contracting,Administraciخ Pݢlica Federal,adquisici material osteosintesis endoprotesis,99100000,IMSS,Ciudad de México,Mexico,MX324,7892.8457,0,NaN,100.0,100.0,NaN,NaN,99.975143,99100000,CPV2008
1231922,1785779,conservas la costena sa de cv,2018,MXN,Adjudicación Directa Federal,goods,MX,NaN,7.387640e+04,NaN,NaN,NaN,NaN,NaN,direct contracting,Administraciخ Pݢlica Federal,chiles jalapenos enteros,99100000,Diconsa S.A. de C.V.,Tonalá,Mexico,MX295,8094.2695,0,NaN,100.0,0.0,NaN,NaN,99.832214,99100000,CPV2008
1231923,1688048,grupo farmacos especializados sa de cv,2018,MXN,Adjudicación Directa Federal,goods,MX,NaN,1.475672e+05,NaN,434.0,NaN,NaN,NaN,direct contracting,Administraciخ Pݢlica Federal,compra medicamentos,99100000,Instituto Nacional de Rehabilitación Luis Guil...,Tlalpan,Mexico,MX325,16168.2050,0,NaN,100.0,0.0,NaN,NaN,99.457687,99100000,CPV2008
1231924,1704492,karla maria rodriguez lastra,2018,MXN,Invitación a Cuando Menos 3 Personas,works,MX,827011008.0,1.967468e+06,NaN,NaN,01jun2018,12jun2018,NaN,invitation (3 entities),Gobierno Municipal,pavimentacion camino base mezcla asfaltica,99300000,Municipal Palace of Villahermosa,Villahermosa,Mexico,MX391,215565.6900,100,NaN,100.0,0.0,50.0,100.0,95.558128,99300000,CPV2008


In [6]:
paraguay_procurement_data = pd.read_csv(os.path.join(data_folder, 'dfid2_py_210715_csv.csv'))
paraguay_procurement_data

,tender_id,tender_title,tender_proceduretype,tender_nationalproceduretype,tender_supplytype,tender_recordedbidscount,lot_productCode,tender_awarddecisiondate,tender_finalprice,bid_price,lot_title,lot_bidscount,buyer_id,buyer_masterid,buyer_name,buyer_geocodes,buyer_city,buyer_country,buyer_buyertype,bidder_id,bidder_masterid,bidder_name,bidder_geocodes,bidder_city,bidder_country,bid_iswinning,source,tender_publications_lastcontract,tender_publications_firstdcontra,notice_url,tender_year,tender_indicator_integrity_singl,tender_indicator_integrity_adver,tender_indicator_integrity_decis,tender_indicator_integrity_call_,tender_indicator_integrity_proce,tender_indicator_integrity_tax_h,tender_integrity_winner_ca_share,currency,tender_biddeadline,tender_contractsignaturedate,tender_publications_firstcallfor,tender_publications_lastcallfort,lot_localProductCode,lot_localProductCode_type
0,0000d077-12aa-48e5-8393-1f8684a17820,Adquisición de vehículos y maquinarias para la...,NaN,Licitación Pública Nacional,NaN,3,"25101507-002,",NaN,1.272480e+09,1.724800e+08,Adquisición de vehículos y maquinarias para la...,NaN,NaN,5e46b4f7-3b99-4e5f-ad7f-3dcc351262c7,Gobierno Departamental de Cordillera,NaN,NaN,NaN,NaN,80048600-5,819bbfb3-bacc-427e-946f-0d9b7ba56f8c,MOTORMARKET S.A.,NaN,FERNANDO DE LA MORA,PY,t,https://www.contrataciones.gov.py,https://www.contrataciones.gov.py:443/datos/ap...,2017-10-23,https://www.contrataciones.gov.py:443/datos/ap...,2017,NaN,NaN,NaN,100,NaN,NaN,NaN,PYG,21088.0,21137.0,21047.0,21066.0,25101507.0,CPV2008
1,0000d077-12aa-48e5-8393-1f8684a17820,Adquisición de vehículos y maquinarias para la...,NaN,Licitación Pública Nacional,NaN,3,"25181709-004,",NaN,1.272480e+09,1.100000e+09,Adquisición de vehículos y maquinarias para la...,NaN,NaN,5e46b4f7-3b99-4e5f-ad7f-3dcc351262c7,Gobierno Departamental de Cordillera,NaN,NaN,NaN,NaN,80002612-8,368896ea-2108-493c-815b-fbefc28b7b4b,RIEDER Y CIA. SACI,NaN,ASUNCION (DISTRITO),PY,t,https://www.contrataciones.gov.py,https://www.contrataciones.gov.py:443/datos/ap...,2017-10-23,https://www.contrataciones.gov.py:443/datos/ap...,2017,NaN,NaN,NaN,100,NaN,NaN,NaN,PYG,21088.0,21137.0,21047.0,21066.0,25181709.0,CPV2008
2,0000e371-dca5-4760-8c4e-93fd4f82bcf7,"ALQUILER DE MAQUINARIAS Y EQUIPOS, DE COMPUTAC...",NaN,Contratación Directa,NaN,1,"80161504-001,",NaN,6.300000e+06,6.300000e+06,"ALQUILER DE MAQUINARIAS Y EQUIPOS, DE COMPUTAC...",NaN,NaN,23b37914-6261-48f1-9e72-409adeff2e59,Facultad de Ciencias Economicas / Universidad ...,NaN,NaN,NaN,NaN,3358978-0,da7fbca3-024f-4b9e-b824-635fac3cae6d,DANIEL ELIAS SERVIN CESPEDES,NaN,PILAR,PY,t,https://www.contrataciones.gov.py,https://www.contrataciones.gov.py:443/datos/ap...,2011-07-12,https://www.contrataciones.gov.py:443/datos/ap...,2011,NaN,NaN,NaN,100,NaN,NaN,NaN,PYG,18785.0,18820.0,18777.0,18777.0,80161504.0,CPV2008
3,0000f14e-0fae-44de-99a8-fe84adbce19c,Seguro para Vehiculos,NaN,Contratación Directa,NaN,1,"84131503-001,",NaN,6.600000e+06,6.600000e+06,Seguro para Vehiculos,NaN,NaN,3f2429eb-1fe6-46d1-94ca-b34c80e3eae6,Facultad de Ciencias Economicas / Universidad ...,NaN,NaN,NaN,NaN,80031893-5,d690b1f3-8193-41e9-bf59-1bd45fa1b185,ASEGURADORA DEL SUR SA SEGUROS GENERALES (ASUR),NaN,ASUNCION (DISTRITO),PY,t,https://www.contrataciones.gov.py,https://www.contrataciones.gov.py:443/datos/ap...,2018-04-19,https://www.contrataciones.gov.py:443/datos/ap...,2018,NaN,NaN,NaN,100,NaN,NaN,NaN,PYG,21264.0,21293.0,21210.0,21257.0,84131503.0,CPV2008
4,00020730-a5d5-4427-88d8-df38f3de5299,CONSTRUCCIÓN DE PAVIMENTO TIPO EMPEDRADO,NaN,Concurso de Ofertas,NaN,1,"72131701-002,72131601-013,72102304-005,8210150...",NaN,4.986200e+08,4.986200e+08,CONSTRUCCIÓN DE PAVIMENTO TIPO EMPEDRADO,NaN,NaN,c00fd959-35ac-4d46-bb44-25d6f65e573e,Gobierno Departamental de Canindeyú,NaN,NaN,NaN,NaN,2979720-9,a0603bb1-6c78-4e55-b36e-50389cce3506,GENARO ANDRES FARIÑA,NaN,FCO. CABALLERO ALVAREZ,PY,t,https://www.contrataciones.gov.py,https://www.contrataciones.gov.py:443/datos/ap...,2012-08-14,

In [7]:
uruguay_procurement_data = pd.read_csv(os.path.join(data_folder, 'dfid2_uy_210715_csv.csv'))
uruguay_procurement_data

,tender_id,tender_title,tender_nationalproceduretype,tender_supplytype,tender_recordedbidscount,tender_contractsignaturedate,lot_productCode,tender_finalprice,bid_price,lot_title,lot_bidscount,buyer_id,buyer_masterid,buyer_name,buyer_nuts,buyer_city,buyer_country,buyer_buyertype,bidder_id,bidder_masterid,bidder_name,bidder_nuts,bidder_city,bidder_country,bid_iswinning,source,tender_publications_lastcontract,tender_publications_firstdcontra,notice_url,tender_publications_lastcallfort,tender_year,tender_indicator_integrity_singl,tender_indicator_integrity_adver,tender_indicator_integrity_decis,tender_indicator_integrity_call_,tender_indicator_integrity_proce,tender_indicator_integrity_tax_h,currency,tender_proceduretype,tender_biddeadline,tender_awarddecisiondate,tender_publications_firstcallfor,lot_localProductCode,lot_localProductCode_type,tender_integrity_winner_ca_share
0,UY_abd1516f6b2b558de01ed1989aa7ed05edc9282f_1,Licitación Pública 7/2006,Licitación Pública,NaN,1,NaN,"35905,35902,1879,1880,1133,",NaN,NaN,R/210000700010,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/210000700010,00e039fb-71dc-4300-930e-0c69586a08a9,MAPA S.A.,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2007-02-07,NaN,NaN,2006,0.0,0.0,50.0,100,100.0,NaN,NaN,OPEN,17097.0,17204.0,NaN,"35905,35",CPV2008,NaN
1,UY_4939b253f303e7e80ca6747952cafbcf6e1b3d01_1,Licitación Abreviada 51/2010,Licitación Abreviada,NaN,1,NaN,"15058,15323,",NaN,NaN,R/210276500016,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/210276500016,9761d34b-9a31-4e73-add1-1df39efb5934,PROVIMAR LTDA.,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2010-12-03,NaN,NaN,2010,0.0,0.0,50.0,100,100.0,NaN,NaN,OPEN,18578.0,18599.0,NaN,"15058,15",CPV2008,NaN
2,UY_1314810c6d1f010c6123f97ae8be72ef29e884dd_1,Licitación Abreviada 10/2011,Licitación Abreviada,NaN,1,NaN,"13370,118,304,8336,185,10838,16,23,8482,6679,1...",NaN,NaN,R/214874940012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/214874940012,f7e1bf2e-7644-4b3f-9db7-9135bff8899e,PAPELCUR SOCIEDAD ANONIMA,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2012-10-11,NaN,2012-03-12,2012,0.0,0.0,100.0,100,100.0,NaN,NaN,OPEN,19079.0,19277.0,19064.0,"13370,11",CPV2008,NaN
3,UY_ddbbcd4eb74f0a0b15fec0091738bd3f5977dc2d_1,Licitación Pública 10/2006,Licitación Pública,NaN,1,NaN,"14022,29369,148,",NaN,NaN,R/211603980013,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/211603980013,8bf14c4b-b8ba-4d1c-a1a9-6f87e544a5cf,DISTRIBUIDORA SANTA ANA S A,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2006-05-16,NaN,NaN,2006,0.0,0.0,50.0,100,100.0,NaN,NaN,OPEN,16887.0,16937.0,NaN,"14022,29",CPV2008,NaN
4,UY_b7dad33fc4bbffd6c1d004919f3c82cab54e3d99_1,Licitación Pública 7/2005,Licitación Pública,NaN,1,NaN,"16610,26399,25932,31427,31461,3222,31464,26311...",NaN,NaN,R/211396450014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/211396450014,67c1edf3-b343-4568-91db-a6a987b8c1fb,URUFARMA S.A.,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2006-04-28,NaN,NaN,2005,0.0,0.0,100.0,100,100.0,NaN,NaN,OPEN,16733.0,16919.0,NaN,"16610,26",CPV2008,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945083,UY_fce415971ee3822246a88eb29185c386bfb15739_1,NaN,NaN,NaN,1,NaN,"0,",NaN,NaN,T/99042,1,11-18,259c3f93-7578-4084-8c1d-0b7151ab63a5,DIRECCIÓN GENERAL DE REGISTROS,NaN,NaN,NaN,NaN,R/214130990011,46980efa-aae8-4ec3-8f34-eca498b5effa,ADMINISTRACION NACIONAL DE CORREOS,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2002-05-20,NaN,NaN,2002,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,15399.0,NaN,"0,",CPV2008,99.365898
945084,UY_2e76ea98af0be5415e061cfad40d11d5647e255f_1,Compra Directa 9017/2010,Compra Directa,NaN,1,NaN,"34036,8317,",NaN,NaN,R/210184230012,1,11-18,259c3f93-7578-4084-8c1d-0b7151ab63a5,DIRECCIÓN GENERAL DE REGISTROS,NaN,NaN,NaN,NaN,R/210184230012,7af0c3f2-3608-4629-9bc7-601659f38420,BELLO LTDA.-,NaN,NaN,NaN,t,https://

### The File which Contains the UNSPSC Classifications

In [8]:
unspsc_classifications = pd.read_excel(os.path.join(data_folder, 'UNGM_UNSPSC_01-Oct-2025..xlsx')).dropna(subset = ['Parent key'])
unspsc_classifications

,Key,Parent key,Code,Title
10,100001,1.000000e+02,10000000,Live Plant and Animal Material and Accessories...
11,100002,1.000010e+05,10100000,Live animals
12,100003,1.000020e+05,10101500,Livestock
13,100004,1.000030e+05,10101501,Cats
14,100005,1.000030e+05,10101502,Dogs
...,...,...,...,...
13288,2137002045,4.479407e+08,57030303,Water distribution kit
13289,2138314803,1.769071e+09,57030109,Trauma and emergency surgery kit
13290,2138640862,1.015260e+05,25131511,"Aircraft, Twin Engine"
13291,2140382544,1.075370e+05,25173121,Distance Measuring Equipment (DME) System


# Data Preprocessing

It appears that there are some obvious issues with the Classifications File, as Parent Key is looking like an integer when it is a categorical variable. Will also inspect the code and key variables

In [9]:
unspsc_classifications.dtypes

Key             int64
Parent key    float64
Code           object
Title          object
dtype: object

In [10]:
unspsc_classifications['Parent key'] = unspsc_classifications['Parent key'].astype(int)
unspsc_classifications['Parent key'] = unspsc_classifications['Parent key'].astype(str)
unspsc_classifications['Key'] = unspsc_classifications['Key'].astype(str)

In [11]:
unspsc_classifications.dtypes

Key           object
Parent key    object
Code          object
Title         object
dtype: object

In [12]:
unspsc_classifications

,Key,Parent key,Code,Title
10,100001,100,10000000,Live Plant and Animal Material and Accessories...
11,100002,100001,10100000,Live animals
12,100003,100002,10101500,Livestock
13,100004,100003,10101501,Cats
14,100005,100003,10101502,Dogs
...,...,...,...,...
13288,2137002045,447940749,57030303,Water distribution kit
13289,2138314803,1769071149,57030109,Trauma and emergency surgery kit
13290,2138640862,101526,25131511,"Aircraft, Twin Engine"
13291,2140382544,107537,25173121,Distance Measuring Equipment (DME) System


### Looking at the different variables of each dataset

The variables the final dataset in the paper included information such as:
- procedure type for tendering 
- number of received bids
- names of buyers and bidders
- tender notice dates
- tender deadlines
- tender award decision dates

In [13]:
mexico_procurement_data.columns

Index(['tender_id', 'bidder_name', 'tender_year', 'currency',
       'tender_nationalproceduretype', 'tender_supplytype', 'bidder_country',
       'buyer_id', 'bid_price', 'tender_recordedbidscount', 'bidder_id',
       'tender_publications_firstcallfor', 'tender_biddeadline',
       'tender_contractsignaturedate', 'tender_proceduretype',
       'buyer_buyertype', 'tender_title', 'lot_productCode', 'buyer_name',
       'buyer_city', 'buyer_country', 'buyer_geocodes', 'bid_priceUsd',
       'tender_indicator_integrity_call_', 'tender_indicator_integrity_singl',
       'tender_indicator_integrity_tax_h', 'tender_indicator_integrity_proce',
       'tender_indicator_integrity_adver', 'tender_indicator_integrity_decis',
       'tender_integrity_winner_ca_share', 'lot_localProductCode',
       'lot_localProductCode_type'],
      dtype='object')

Names of the columns suggesting each of the above preprocessing steps:
- `tender_proceduretype`
- `tender_recordedbidscount`
- `buyer_name` and `bidder_name`
- `tender_publications_firstcallfor`
- `tender_biddeadline`
- `tender_contractsignaturedate`

In [14]:
paraguay_procurement_data.columns

Index(['tender_id', 'tender_title', 'tender_proceduretype',
       'tender_nationalproceduretype', 'tender_supplytype',
       'tender_recordedbidscount', 'lot_productCode',
       'tender_awarddecisiondate', 'tender_finalprice', 'bid_price',
       'lot_title', 'lot_bidscount', 'buyer_id', 'buyer_masterid',
       'buyer_name', 'buyer_geocodes', 'buyer_city', 'buyer_country',
       'buyer_buyertype', 'bidder_id', 'bidder_masterid', 'bidder_name',
       'bidder_geocodes', 'bidder_city', 'bidder_country', 'bid_iswinning',
       'source', 'tender_publications_lastcontract',
       'tender_publications_firstdcontra', 'notice_url', 'tender_year',
       'tender_indicator_integrity_singl', 'tender_indicator_integrity_adver',
       'tender_indicator_integrity_decis', 'tender_indicator_integrity_call_',
       'tender_indicator_integrity_proce', 'tender_indicator_integrity_tax_h',
       'tender_integrity_winner_ca_share', 'currency', 'tender_biddeadline',
       'tender_contractsignature

Names of the columns suggesting each of the above preprocessing steps:
- `tender_proceduretype`
- `tender_recordedbidscount`
- `buyer_name` and `bidder_name`
- `tender_publications_firstcallfor`
- `tender_biddeadline`
- `tender_contractsignaturedate`

In [15]:
uruguay_procurement_data.columns

Index(['tender_id', 'tender_title', 'tender_nationalproceduretype',
       'tender_supplytype', 'tender_recordedbidscount',
       'tender_contractsignaturedate', 'lot_productCode', 'tender_finalprice',
       'bid_price', 'lot_title', 'lot_bidscount', 'buyer_id', 'buyer_masterid',
       'buyer_name', 'buyer_nuts', 'buyer_city', 'buyer_country',
       'buyer_buyertype', 'bidder_id', 'bidder_masterid', 'bidder_name',
       'bidder_nuts', 'bidder_city', 'bidder_country', 'bid_iswinning',
       'source', 'tender_publications_lastcontract',
       'tender_publications_firstdcontra', 'notice_url',
       'tender_publications_lastcallfort', 'tender_year',
       'tender_indicator_integrity_singl', 'tender_indicator_integrity_adver',
       'tender_indicator_integrity_decis', 'tender_indicator_integrity_call_',
       'tender_indicator_integrity_proce', 'tender_indicator_integrity_tax_h',
       'currency', 'tender_proceduretype', 'tender_biddeadline',
       'tender_awarddecisiondate', '

Names of the columns suggesting each of the above preprocessing steps:
- `tender_proceduretype`
- `tender_recordedbidscount`
- `buyer_name` and `bidder_name`
- `tender_publications_firstcallfor`
- `tender_biddeadline`
- `tender_contractsignaturedate`

## Now Ensuring that Each Column Has the Information Required by the Paper

In [16]:
print_columns_check(mexico_procurement_data, "Mexico")

Procedure Types for Tenders in Mexico: tender_nationalproceduretype
Adjudicación Directa Federal            860217
Licitación Pública                      190859
Invitación a Cuando Menos 3 Personas    148969
Adjudicación directa                       838
Licitación Pública con OSD                 390
Proyecto de Convocatoria                   321
Otro                                       149
Licitación Publica Estatal                   5
Name: count, dtype: int64

Number of Recorded Bids in Mexico: tender_recordedbidscount
1.0    11757
Name: count, dtype: int64

Number of Buyers in Mexico:  294

Number of Bidders in Mexico:  185702

Tender Publication Call Dates in Mexico:  tender_publications_firstcallfor
14feb2017    1735
09feb2017    1708
16feb2017    1642
03jun2016    1630
19apr2017    1623
             ... 
18aug2011       1
15aug2011       1
07jan2017       1
08feb2011       1
27may2018       1
Name: count, Length: 2617, dtype: int64

Tender Bid Deadlines in Mexico:  tender_bid

In [17]:
print_columns_check(paraguay_procurement_data, 'Paraguay')

Procedure Types for Tenders in Paraguay: tender_nationalproceduretype
Contratación Directa                                                                                90265
Concurso de Ofertas                                                                                 32752
Licitación Pública Nacional                                                                         30365
Locación de Inmuebles                                                                                5121
Contratación por Excepción                                                                           4399
Licitación Pública Internacional                                                                      858
BID - Contratación Directa (CD/SD)                                                                    571
BID - Concurso de Ofertas (CO/CP)                                                                     295
BID - Licitación Internacional                                                    

In [18]:
print_columns_check(uruguay_procurement_data, "Uruguay")

Procedure Types for Tenders in Uruguay: tender_nationalproceduretype
Compra Directa                              275987
Licitación Abreviada                         63199
Licitación Pública                            9232
Compra por Excepción                          6601
Venta/Arrendamiento por Remate                 541
Procedimiento Especial                         367
PFI - Comparación de precios                   267
Concesión                                       99
Pregón                                          66
Venta/Arrendamiento Licitación Abreviada        45
Venta/Arrendamiento Directa                     30
PFI - Licitación pública internacional          22
PFI - Licitación pública nacional               16
PFI - Contratación directa                      14
Venta/Arrendamiento Licitación Pública           4
Name: count, dtype: int64

Number of Recorded Bids in Uruguay: tender_recordedbidscount
1    945088
Name: count, dtype: int64

Number of Buyers in Uruguay:  357

Numb

### Checking All Product Codes to Match Against UNSPSC
<a href="https://www.ungm.org/public/unspsc">Here is the Product Code matching they use</a>

Need to create a dictionary object of all line items that have pharmaceutical related observations, then compare them against what is found in the main country datasets

In [19]:
unspsc_classifications.columns = ['Key', 'Parent key', 'Product Code', 'Title']
unspsc_classifications

,Key,Parent key,Product Code,Title
10,100001,100,10000000,Live Plant and Animal Material and Accessories...
11,100002,100001,10100000,Live animals
12,100003,100002,10101500,Livestock
13,100004,100003,10101501,Cats
14,100005,100003,10101502,Dogs
...,...,...,...,...
13288,2137002045,447940749,57030303,Water distribution kit
13289,2138314803,1769071149,57030109,Trauma and emergency surgery kit
13290,2138640862,101526,25131511,"Aircraft, Twin Engine"
13291,2140382544,107537,25173121,Distance Measuring Equipment (DME) System


In [20]:
pharmaceutical_classifications = unspsc_classifications[unspsc_classifications['Title'].str.contains('pharma|drug|medicine|medical|health')]
pharmaceutical_classifications

,Key,Parent key,Product Code,Title
821,102545,102525,95141900,Prefabricated medical buildings and structures
2306,106382,106358,51300000,Antifungal drugs
2324,106409,181731,51281500,Antitubercular drugs
2334,106428,106358,51340000,Antiviral drugs
2382,106512,106358,51120000,Antiarrythmics and antianginals and cardiopleg...
...,...,...,...,...
12812,789763802,1769071149,57030107,Primary healthcare kit
12924,1097150747,115471,42192800,Medical and pharmaceutical biohazard disposal ...
13118,1654384682,1769071149,57030105,Emergency health kit
13127,1667187086,1699154932,57030000,Humanitarian health supplies


In [21]:
mexico_procurement_data = mexico_procurement_data.rename(columns = {'lot_productCode' : 'Product Code'})

In [22]:
unspsc_classifications_mapped = unspsc_classifications.set_index('Product Code')['Title'].to_dict()
unspsc_classifications_mapped

{'10000000': 'Live Plant and Animal Material and Accessories and Supplies',
 '10100000': 'Live animals',
 '10101500': 'Livestock',
 '10101501': 'Cats',
 '10101502': 'Dogs',
 '10101506': 'Horses',
 '10101507': 'Sheep',
 '10101508': 'Goats',
 '10101509': 'Asses',
 '10101511': 'Swine',
 '10101512': 'Rabbits',
 '10101513': 'Guinea pigs',
 '10101516': 'Cattle',
 '10101600': 'Birds and fowl',
 '10101601': 'Live chickens',
 '10101602': 'Live ducks',
 '10101603': 'Live turkeys',
 '10101604': 'Live geese',
 '10101605': 'Live pheasants',
 '10101700': 'Live fish',
 '10101701': 'Live salmon',
 '10101800': 'Shellfish and aquatic invertebrates',
 '10101801': 'Live shrimp',
 '10101900': 'Insects',
 '10101903': 'Bees',
 '10101904': 'Silkworms',
 '10120000': 'Animal feed',
 '10121500': 'Livestock feed',
 '10121501': 'Pure wheat bran',
 '10121600': 'Bird and fowl food',
 '10121601': 'Live food for birds',
 '10121700': 'Fish food',
 '10121701': 'Fresh or frozen brine',
 '10121800': 'Dog and cat food',
 '

In [23]:
mexico_procurement_data['Product Classification'] = mexico_procurement_data['Product Code'].map(unspsc_classifications_mapped)


In [24]:
mexico_procurement_data['Product Code'].value_counts().to_dict()

{99100000: 483759,
 99200000: 346726,
 99300000: 113005,
 330000000: 57516,
 398000000: 22699,
 450000007: 13987,
 301900007: 10446,
 726000006: 10111,
 796200006: 9287,
 349000006: 8776,
 310000006: 8594,
 452310005: 8380,
 324000007: 7305,
 301927008: 6888,
 411100003: 6107,
 302000001: 5819,
 805210002: 5740,
 907221005: 5711,
 358000002: 5496,
 800000004: 5329,
 501000006: 4934,
 600000008: 4697,
 150000008: 4532,
 320000003: 4089,
 343500005: 3906,
 425100004: 3510,
 480000008: 3328,
 454000001: 3247,
 9100000: 2909,
 798100005: 2802,
 9120000: 2645,
 228000008: 2181,
 9134100: 2105,
 146220007: 2069,
 660000000: 1995,
 793400009: 1925,
 331510003: 1779,
 482000000: 1722,
 421200006: 1702,
 661710009: 1701,
 441631128: 1692,
 223000003: 1547,
 249511006: 1532,
 430000003: 1511,
 300000009: 1499,
 452130003: 1498,
 454300000: 1348,
 909100009: 1327,
 905110002: 1323,
 311000007: 1222,
 904000001: 1132,
 722000007: 1104,
 441000001: 1068,
 391100006: 876,
 791000005: 753,
 301000000

Preparing the codes for Paraguay Products

In [25]:
paraguay_product_codes = paraguay_procurement_data['lot_productCode'].value_counts().reset_index()
paraguay_product_codes['lot_productCode'] = paraguay_product_codes['lot_productCode'].str.strip(',')

paraguay_product_codes = paraguay_product_codes.assign(lot_productCode = paraguay_product_codes['lot_productCode'].str.split(','))
paraguay_product_codes = paraguay_product_codes.explode('lot_productCode')

paraguay_product_base_codes = paraguay_product_codes.copy()

paraguay_product_base_codes['lot_productCode'] = paraguay_product_base_codes['lot_productCode'].str.split('-').str[0]

paraguay_product_codes = paraguay_product_codes.groupby('lot_productCode', as_index = False)['count'].sum()
paraguay_product_base_codes = paraguay_product_base_codes.groupby('lot_productCode', as_index = False)['count'].sum()
paraguay_product_base_codes

,lot_productCode,count
0,10101502,3
1,10101506,13
2,10101507,6
3,10101508,6
4,10101511,11
...,...,...
5840,95121911,2
5841,95131603,1
5842,95131605,2
5843,95131702,1


In [26]:
paraguay_product_base_codes

,lot_productCode,count
0,10101502,3
1,10101506,13
2,10101507,6
3,10101508,6
4,10101511,11
...,...,...
5840,95121911,2
5841,95131603,1
5842,95131605,2
5843,95131702,1


In [27]:
paraguay_product_base_codes.columns = ['Product Code', 'count']
paraguay_product_base_codes

,Product Code,count
0,10101502,3
1,10101506,13
2,10101507,6
3,10101508,6
4,10101511,11
...,...,...
5840,95121911,2
5841,95131603,1
5842,95131605,2
5843,95131702,1


In [28]:
paraguay_products_titled = pd.merge(paraguay_product_base_codes, pharmaceutical_classifications, how = 'inner', on = 'Product Code')
len(sorted(paraguay_products_titled['Title'].value_counts().to_dict()))

15

In [29]:
uruguay_procurement_data

,tender_id,tender_title,tender_nationalproceduretype,tender_supplytype,tender_recordedbidscount,tender_contractsignaturedate,lot_productCode,tender_finalprice,bid_price,lot_title,lot_bidscount,buyer_id,buyer_masterid,buyer_name,buyer_nuts,buyer_city,buyer_country,buyer_buyertype,bidder_id,bidder_masterid,bidder_name,bidder_nuts,bidder_city,bidder_country,bid_iswinning,source,tender_publications_lastcontract,tender_publications_firstdcontra,notice_url,tender_publications_lastcallfort,tender_year,tender_indicator_integrity_singl,tender_indicator_integrity_adver,tender_indicator_integrity_decis,tender_indicator_integrity_call_,tender_indicator_integrity_proce,tender_indicator_integrity_tax_h,currency,tender_proceduretype,tender_biddeadline,tender_awarddecisiondate,tender_publications_firstcallfor,lot_localProductCode,lot_localProductCode_type,tender_integrity_winner_ca_share
0,UY_abd1516f6b2b558de01ed1989aa7ed05edc9282f_1,Licitación Pública 7/2006,Licitación Pública,NaN,1,NaN,"35905,35902,1879,1880,1133,",NaN,NaN,R/210000700010,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/210000700010,00e039fb-71dc-4300-930e-0c69586a08a9,MAPA S.A.,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2007-02-07,NaN,NaN,2006,0.0,0.0,50.0,100,100.0,NaN,NaN,OPEN,17097.0,17204.0,NaN,"35905,35",CPV2008,NaN
1,UY_4939b253f303e7e80ca6747952cafbcf6e1b3d01_1,Licitación Abreviada 51/2010,Licitación Abreviada,NaN,1,NaN,"15058,15323,",NaN,NaN,R/210276500016,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/210276500016,9761d34b-9a31-4e73-add1-1df39efb5934,PROVIMAR LTDA.,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2010-12-03,NaN,NaN,2010,0.0,0.0,50.0,100,100.0,NaN,NaN,OPEN,18578.0,18599.0,NaN,"15058,15",CPV2008,NaN
2,UY_1314810c6d1f010c6123f97ae8be72ef29e884dd_1,Licitación Abreviada 10/2011,Licitación Abreviada,NaN,1,NaN,"13370,118,304,8336,185,10838,16,23,8482,6679,1...",NaN,NaN,R/214874940012,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/214874940012,f7e1bf2e-7644-4b3f-9db7-9135bff8899e,PAPELCUR SOCIEDAD ANONIMA,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2012-10-11,NaN,2012-03-12,2012,0.0,0.0,100.0,100,100.0,NaN,NaN,OPEN,19079.0,19277.0,19064.0,"13370,11",CPV2008,NaN
3,UY_ddbbcd4eb74f0a0b15fec0091738bd3f5977dc2d_1,Licitación Pública 10/2006,Licitación Pública,NaN,1,NaN,"14022,29369,148,",NaN,NaN,R/211603980013,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/211603980013,8bf14c4b-b8ba-4d1c-a1a9-6f87e544a5cf,DISTRIBUIDORA SANTA ANA S A,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2006-05-16,NaN,NaN,2006,0.0,0.0,50.0,100,100.0,NaN,NaN,OPEN,16887.0,16937.0,NaN,"14022,29",CPV2008,NaN
4,UY_b7dad33fc4bbffd6c1d004919f3c82cab54e3d99_1,Licitación Pública 7/2005,Licitación Pública,NaN,1,NaN,"16610,26399,25932,31427,31461,3222,31464,26311...",NaN,NaN,R/211396450014,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R/211396450014,67c1edf3-b343-4568-91db-a6a987b8c1fb,URUFARMA S.A.,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2006-04-28,NaN,NaN,2005,0.0,0.0,100.0,100,100.0,NaN,NaN,OPEN,16733.0,16919.0,NaN,"16610,26",CPV2008,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945083,UY_fce415971ee3822246a88eb29185c386bfb15739_1,NaN,NaN,NaN,1,NaN,"0,",NaN,NaN,T/99042,1,11-18,259c3f93-7578-4084-8c1d-0b7151ab63a5,DIRECCIÓN GENERAL DE REGISTROS,NaN,NaN,NaN,NaN,R/214130990011,46980efa-aae8-4ec3-8f34-eca498b5effa,ADMINISTRACION NACIONAL DE CORREOS,NaN,NaN,NaN,t,https://catalogodatos.gub.uy/dataset/acce-dato...,NaN,2002-05-20,NaN,NaN,2002,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,15399.0,NaN,"0,",CPV2008,99.365898
945084,UY_2e76ea98af0be5415e061cfad40d11d5647e255f_1,Compra Directa 9017/2010,Compra Directa,NaN,1,NaN,"34036,8317,",NaN,NaN,R/210184230012,1,11-18,259c3f93-7578-4084-8c1d-0b7151ab63a5,DIRECCIÓN GENERAL DE REGISTROS,NaN,NaN,NaN,NaN,R/210184230012,7af0c3f2-3608-4629-9bc7-601659f38420,BELLO LTDA.-,NaN,NaN,NaN,t,https://

# Loading in Each Countries Procurement Catalogs

## Searching the Paraguayan Catalog
From previous inspection of the dataset, we only need to take the n4 codes. Everything else returns an error when searching the Paraguayan tender dataset

In [30]:
paraguay_public_procurement_catalog = pd.read_csv(paraguayan_procurement_catalog_file_path, 
                                                               usecols = ['id', 
                                                                          'n4_codigo', 
                                                                          'n4_nombre', 
                                                                          'nombre',
                                                                          'createdDate',
                                                                          'date'])
paraguay_public_procurement_num_rows = paraguay_public_procurement_catalog.shape[0]
paraguay_public_procurement_num_cols = paraguay_public_procurement_catalog.shape[1]

print(f'{paraguay_public_procurement_num_rows} rows x {paraguay_public_procurement_num_cols} columns')
paraguay_public_procurement_catalog.head()


307 rows x 6 columns


,id,n4_codigo,n4_nombre,nombre,createdDate,date
0,14111510-9997,14111510,Papeles especiales,Papel para encuadernación,2025-06-12T14:30:08-04:00,2025-10-02T21:46:46-04:00
1,15101506-9996,15101506,Gasolina,Gasolina sin plomo Ron 88,2025-04-15T11:30:58-04:00,2025-10-02T21:46:46-04:00
2,30141508-9999,30141508,Aislamiento de mica,Cinta de terminación sin adhesivo,2025-06-02T16:08:44-04:00,2025-10-02T21:46:48-04:00
3,31211803-9988,31211803,Disolventes y Diluyentes para pinturas y barnices,Thinner de 18 lts,2025-06-27T08:54:52-04:00,2025-10-02T21:46:49-04:00
4,39111503-9998,39111503,Dispositivos de pared,Panel Led para adosar de 12 W.,2025-05-30T14:47:02-04:00,2025-10-02T21:46:49-04:00


Running the Translations

In [ ]:
paraguay_drop_cols = ['id', 'n4_codigo', 'createdDate', 'date']

paraguay_public_procurement_items_translated = translate_country_procurement_catalog(
    paraguay_public_procurement_catalog,
    paraguayan_procurement_catalog_file_name,
    'csv',
    paraguay_drop_cols)

File Not Found, Running Translation


KeyError: "['n4_codigocreatedDate'] not found in axis"

## Searching the Uruguayan Catalog 

Data Dictionary

In [ ]:
uruguay_public_procurement_catalog_data_dict = pd.read_excel(os.path.join(data_folder, 'diccionario_datos_rupe.xlsx'))
uruguay_public_procurement_catalog_data_dict

,Nombre del campo,Descripción,Tipo de datos
0,pais_prov,País del proveedor,Texto
1,identificacion_prov,Identificación del proveedor,Texto
2,denominacion_social_prov,Denominación social del proveedor,Texto
3,domicilio_fiscal,Domicilio fiscal del proveedor,Texto
4,localidad_prov,Localidad del proveedor,Texto
5,departamento_prov,Departamento del proveedor,Texto
6,estado_prov,Estado en que se encuentra el registro del pro...,Texto


Translating the `Descripción` column

In [ ]:
uruguay_public_procurement_catalog_data_dict['Descripción'] = uruguay_public_procurement_catalog_data_dict['Descripción'].apply(lambda x: translator.translate(x))
uruguay_public_procurement_catalog_data_dict

,Nombre del campo,Descripción,Tipo de datos
0,pais_prov,Supplier country,Texto
1,identificacion_prov,Supplier identification,Texto
2,denominacion_social_prov,Social name of the supplier,Texto
3,domicilio_fiscal,Fiscal Address of the Supplier,Texto
4,localidad_prov,SUPPLIER LOCATION,Texto
5,departamento_prov,Supplier Department,Texto
6,estado_prov,State in which the supplier's registration is ...,Texto


### Now opening the Uruguay Procurement Dataset
The file below can be found <a href="https://www.comprasestatales.gub.uy/sicepublic/SearchCatalogPublic.iface">here</a>
<br>There will be a link to download an excel file format

In [ ]:
uruguay_public_procurement_catalog = pd.read_excel(uruguayan_procurement_catalog_file_path, header = 1)
uruguay_public_procurement_catalog

,Cód. Artículo,Artículo,Familia,SubFamilia,Clase,SubClase,Unidad,Variante,Unidad Variante,Color,Tipo Artículo,Tipo Detalle Variante,ODG,Impuesto
0,108453,. RESISTENCIA AL CONTACTO CON H2S Y AMINA,PRODUCTOS EXCLUIDOS DEL CATALOGO UNICO DE BIEN...,"PRODUCTOS DE USO EXCLUSIVO DE ANCAP, SIN CRITE...",ANCAP,ANCAP,UNIDAD,-----,-----,NaN,Artículo,Sin Detalle,198000,IVA BASICO
1,108894,"ABAC CONJ BLOQ 5/8"" 6000# SS EMPAQ. PTFE",PRODUCTOS EXCLUIDOS DEL CATALOGO UNICO DE BIEN...,"PRODUCTOS DE USO EXCLUSIVO DE ANCAP, SIN CRITE...",ANCAP,ANCAP,UNIDAD,-----,-----,NaN,Artículo,Sin Detalle,173000,IVA BASICO
2,108895,"ABAC VA150/IT VALV AGU SS276 1/2"" 6000# NPT",PRODUCTOS EXCLUIDOS DEL CATALOGO UNICO DE BIEN...,"PRODUCTOS DE USO EXCLUSIVO DE ANCAP, SIN CRITE...",ANCAP,ANCAP,UNIDAD,-----,-----,NaN,Artículo,Sin Detalle,173000,IVA BASICO
3,108896,"ABAC VA250/IT-SG VALV AGU 1/2"" #6000 SS316 NPT",PRODUCTOS EXCLUIDOS DEL CATALOGO UNICO DE BIEN...,"PRODUCTOS DE USO EXCLUSIVO DE ANCAP, SIN CRITE...",ANCAP,ANCAP,UNIDAD,-----,-----,NaN,Artículo,Sin Detalle,173000,IVA BASICO
4,108897,"ABAC VE250 VALV ESF 2VIAS 1/2"" NPT",PRODUCTOS EXCLUIDOS DEL CATALOGO UNICO DE BIEN...,"PRODUCTOS DE USO EXCLUSIVO DE ANCAP, SIN CRITE...",ANCAP,ANCAP,UNIDAD,-----,-----,NaN,Artículo,Sin Detalle,173000,IVA BASICO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,60138,ACONDICIONADOR PARA IONOMERO,MATERIALES Y SUMINISTROS,PROD. QUIMICOS Y CONEXOS EXCEPTO MEDIC.Y ANTIS...,"PRODUCTOS QUIMICOS, MEDICINALES Y FARMACEUTICOS",PRODUCTOS QUIMICOS ODONTOLOGICOS,FRASCO,-----,-----,NaN,Artículo,Sin Detalle,154000,IVA BASICO
996,28421,ACONDICIONADOR USO ODONTOLOGICO,MATERIALES Y SUMINISTROS,PROD. QUIMICOS Y CONEXOS EXCEPTO MEDIC.Y ANTIS...,"PRODUCTOS QUIMICOS, MEDICINALES Y FARMACEUTICOS",PRODUCTOS QUIMICOS ODONTOLOGICOS,AVIO,TIPO,-----,NaN,Artículo,Sin Detalle,154000,IVA BASICO
997,100542,ACONDICIONAMIENTO ACUSTICO DE SALA,"CONSTRUCCIONES, MEJORAS Y REPARACIONES EXTRAOR...",REPARACIONES MAYORES Y EXTRAORDINARIAS,DE INMUEBLES,ACONDICIONAMIENTO DE INMUEBLES,UNIDAD,-----,-----,NaN,Servicio,Sin Detalle,382000,IVA BASICO
998,70824,ACONDICIONAMIENTO DE ALERO,"CONSTRUCCIONES, MEJORAS Y REPARACIONES EXTRAOR...",REPARACIONES MAYORES Y EXTRAORDINARIAS,DE INMUEBLES,RECICLAJE DE INMUEBLES,UNIDAD,-----,-----,NaN,Obra,Sin Detalle,382000,IVA BASICO


`Cód. Articulo` refers to the line items found in the main datasets. `Articulo` would hopefully describe the type of good/service being recorded in the dataset. Going to use the Google Translator function to find this information.

### Running a Translation of the Catalog

In [ ]:
uruguay_drop_cols = [
    'Cód. Artículo',
    'Unidad Variante',
    'Color', 
    'ODG'
]

uruguay_public_procurement_items_translated = translate_country_procurement_catalog(uruguay_public_procurement_catalog,
                                                                                    uruguayan_procurement_catalog_file_name, 
                                                                                    'csv',
                                                                                    uruguay_drop_cols)

File Found. Loading Dataset


## Searching the Mexican Procurement Catalog
The file in the cell below can be found <a href="https://comprasmx.buengobierno.gob.mx/datos-abiertos">here</a>
<br>**it is the first file under Catalogs**

In [ ]:
mexico_public_procurement_catalog = pd.read_excel(mexican_procurement_catalog_file_path)
mexico_public_procurement_catalog.head()

,CLAVE CUCoP +,CLAVE CUCoP,DESCRIPCIÓN,UNIDAD DE MEDIDA (sugerida),TIPO DE CONTRATACIÓN,PARTIDA ESPECÍFICA,DESC. PARTIDA ESPECÍFICA,PARTIDA GENÉRICA,DESC. PARTIDA GENÉRICA,CONCEPTO,DESC. CONCEPTO,CAPÍTULO,DESC. CAPÍTULO,FECHA ALTA CUCOP,FECHA MODIFICACIÓN CUCOP
0,12201-0001,12201001,REMUNERACIONES AL PERSONAL EVENTUAL,SERVICIO,SERVICIOS,12201,REMUNERACIONES AL PERSONAL EVENTUAL,1212,SUELDOS BASE AL PERSONAL EVENTUAL,1200,REMUNERACIONES AL PERSONAL DE CARACTER TRANSIT...,1000,SERVICIOS PERSONALES,25/09/2023,NaN
1,14104-0001,14104001,APORTACIONES DE SEGURIDAD SOCIAL CONTRACTUALES,SERVICIO,SERVICIOS,14104,APORTACIONES DE SEGURIDAD SOCIAL CONTRACTUALES,1440,APORTACIONES PARA SEGUROS,1400,SEGURIDAD SOCIAL,1000,SERVICIOS PERSONALES,03/08/2023,22/06/2023
2,14401-0001,14401001,SEGURO DE VIDA DEL PERSONAL CIVIL,SERVICIO,SERVICIOS,14401,CUOTAS PARA EL SEGURO DE VIDA DEL PERSONAL CIVIL,1440,APORTACIONES PARA SEGUROS,1400,SEGURIDAD SOCIAL,1000,SERVICIOS PERSONALES,NaN,NaN
3,14402-0001,14402001,SEGURO DE VIDA DEL PERSONAL MILITAR,SERVICIO,SERVICIOS,14402,CUOTAS PARA EL SEGURO DE VIDA DEL PERSONAL MIL...,1440,APORTACIONES PARA SEGUROS,1400,SEGURIDAD SOCIAL,1000,SERVICIOS PERSONALES,NaN,NaN
4,14403-0001,14403001,SEGURO DE GASTOS MEDICOS DEL PERSONAL CIVIL,SERVICIO,SERVICIOS,14403,CUOTAS PARA EL SEGURO DE GASTOS MÉDICOS DEL PE...,1440,APORTACIONES PARA SEGUROS,1400,SEGURIDAD SOCIAL,1000,SERVICIOS PERSONALES,NaN,NaN


In [ ]:
mexico_drop_cols = [ 
    'CLAVE CUCoP +', 
    'CLAVE CUCoP', 
    'PARTIDA ESPECÍFICA', 
    'PARTIDA GENÉRICA', 
    'CONCEPTO', 
    'CAPÍTULO', 
    'FECHA ALTA CUCOP', 
    'FECHA MODIFICACIÓN CUCOP' 
]

mexico_public_procurement_catalog_translated = translate_country_procurement_catalog(mexico_public_procurement_catalog,
                                                                                     mexican_procurement_catalog_file_name,
                                                                                     'csv',
                                                                                     mexico_drop_cols)

File Found. Loading Dataset


In [ ]:
for column in mexico_public_procurement_catalog.columns:
    print(f'Column: {column} : English Translation: {translator.translate(column)}')
    

Column: CLAVE CUCoP + : English Translation: CUCOP + KEY
Column: CLAVE CUCoP : English Translation: CUCOP KEY
Column: DESCRIPCIÓN : English Translation: DESCRIPTION
Column: UNIDAD DE MEDIDA (sugerida) : English Translation: Unit of measure (suggested)
Column: TIPO DE CONTRATACIÓN : English Translation: Type of hiring
Column: PARTIDA ESPECÍFICA : English Translation: Specific game
Column: DESC. PARTIDA ESPECÍFICA : English Translation: Desc. Specific game
Column: PARTIDA GENÉRICA : English Translation: Generic game
Column: DESC. PARTIDA GENÉRICA : English Translation: Desc. Generic game
Column: CONCEPTO : English Translation: CONCEPT
Column: DESC. CONCEPTO : English Translation: Desc. CONCEPT
Column: CAPÍTULO : English Translation: CHAPTER
Column: DESC. CAPÍTULO : English Translation: Desc. CHAPTER
Column: FECHA ALTA CUCOP : English Translation: CUCOP HIGH DATE
Column: FECHA MODIFICACIÓN CUCOP : English Translation: CUCOP MODIFICATION DATE


In [ ]:
mexico_procurement_data['Product Code'] = mexico_procurement_data['Product Code'].astype(str)

## Finding Where the Quantity Numbers Exist in the Paraguay Dataset

In [ ]:
tender_publications_list = paraguay_procurement_data.iloc[0:10]['tender_publications_lastcontract'].to_list()

for tender in tender_publications_list:
    if 'contract' in tender:
        print(True)
    # Do an API lookup of the URL

True
True
True
True
True
True
True
